# Servo/RC ESC control with RPI PWM

Adafruit product page: <https://learn.adafruit.com/16-channel-pwm-servo-driver/overview>

<div style="text-align:center;"><img src="images/adafruit_products_raspi_pca9685_i2c_bb.jpg"  width="300"/></div>

In [ ]:
!sudo apt-get install python-smbus i2c-tools
!sudo usermod -a -G i2c banshee

In [2]:
!i2cdetect -y 1

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- -- -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: 40 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: 70 -- -- -- -- -- -- --                         


### Documentation ServoKit: <https://github.com/adafruit/Adafruit_CircuitPython_ServoKit>
### Documentation Extra: <https://circuitpython.readthedocs.io/projects/motor/en/latest/api.html#adafruit_motor.servo.Servo>

## Servo Control

Reading in several sources, Servo controls is usually based in PWM signals of 50Hz base and a variable Duty Cycle between (1ms-2ms). But this edges can vary.

```
Duty Cycle

“Duty cycle” is the width of positive pulse (square wave) and a deciding factor for servo’s angular position. For example, if you have a servo with 180° turn, then 90° is the center position of the servo with 0° being minimum, and 180°, being the maximum. Now, if a positive pulse of 1.5ms is sent, then the servo stays at 90° (servo center) as long as it receives the same pulse. If another pulse of 1ms is sent, the circuit tries to move the shaft to 0°, and a pulse of 2ms tries to move the output shaft to 180°.  This means, a pulse shorter than 1.5ms moves the servo in one direction and wider than 1.5ms moves it in another direction.

Different servo models have different minimum and maximum pulse requirements. For example, a Hextronik servo I have has a minimum pulse requirement of 0.5ms to move to 0° and maximum pulse duration of 2.5ms to move to 180°. Sending a pulse of 1ms moves it to 45° and 2ms moves it to 135°. Another servo requires 1ms pulse to move to 0°, 1.5ms to move to 45° and 2ms to move to 90° and maximum angular rotation being 90°. The de-facto standard is 1ms for minimal angle, 1.5ms for servo center and 2ms for maximum angle. Servo center is almost always 1.5ms and minimum and maximum should be verified in product’s datasheet.
```

It's neccesary identify the limits for every particular servo. Based on engine docmentation if available or manually decrasing min limit or increasing max limits with little steps and checking if engine reaches the physical limit.


In [1]:
import time
from adafruit_servokit import ServoKit

# Set channels to the number of servo channels on your kit.
# 8 for FeatherWing, 16 for Shield/HAT/Bonnet.
kit = ServoKit(channels=16)

In [21]:
kit.servo[8].angle = 0
time.sleep(1)
kit.servo[8].angle = 90
time.sleep(1)
kit.servo[8].angle = 180

<p float="left">
  <img src="images/pwm_angle_0.jpg" alt="angle 0" style="width:30%">
  <img src="images/pwm_angle_90.jpg" alt="angle 0" style="width:30%">
  <img src="images/pwm_angle_180.jpg" alt="angle 0" style="width:30%">
</p>

## Test Servo 

### Model: Carson CS-6

**Specs**:
- ball bearings 
- operating voltage 4.8 - 6 V.
- Actuating torque 60 Ncm
- Adjustment time 0.2 sec / 60 °
- Dimensions: 41 x 42 x 21 mm
- Weight 58 g
- Graupner-J / R connector system

Library:
[Servo Kit Library Documentation](https://circuitpython.readthedocs.io/projects/motor/en/latest/api.html#adafruit_motor.servo.Servo)

Servo limits has been searched and tested with values:
```
ACTUATION_RANGE = 180
MIN_PULSE       = 600   (ms)
MAX_PULSE       = 2400  (ms)
```

In [4]:
from adafruit_servokit import ServoKit
import time
import ipywidgets as widgets
from traitlets import CInt, link

SERVO_CHANNEL = 8
ACTUATION_RANGE = 180
MIN_PULSE = 600             # Default 1000
MAX_PULSE = 2400            # Default 2000

class Counter(widgets.DOMWidget):
    #value = CInt(0, sync=True)
    value = CInt(90)
    
# Value Widget to Sync Sliders and buttons
counterServo = Counter()    

# Servo Kit
kit = ServoKit(channels=16)

# Customize servo range angle
kit.servo[SERVO_CHANNEL].actuation_range = ACTUATION_RANGE

# Pulse Width range
kit.servo[SERVO_CHANNEL].set_pulse_width_range(MIN_PULSE, MAX_PULSE)

# Center Servo
kit.servo[SERVO_CHANNEL].angle = 90

# GUI
style = {'description_width': 'initial'}
servo_duty_widget = widgets.FloatSlider(description='Servo Position', style=style, min=0, max=180, step=1, value=90, layout=widgets.Layout(width='50%', height='30px'))
#servo_duty_widget = widgets.FloatSlider(description='Servo Position', style=style, min=65, max=115, step=1, value=90, layout=widgets.Layout(width='50%', height='30px'))
servo_reset_widget = widgets.Button(description='Servo Reset')
output = widgets.Output()

def on_servo_duty_value_change(change):
    #with output:
        angle = change['new']
        kit.servo[SERVO_CHANNEL].angle = angle
        counterServo.value = angle
        print('Set Aggle : '+str(angle))
        
servo_duty_widget.observe(on_servo_duty_value_change, names='value')

def on_reset_servo_clicked(b):
    #with output:
        counterServo.value = 90
        print("Servo Reset.")
        
servo_reset_widget.on_click(on_reset_servo_clicked) 

# Link Slider and counter
link((servo_duty_widget, 'value'), (counterServo, 'value')) # Link counter al slider

engine_widget = widgets.VBox([
    servo_duty_widget,
    servo_reset_widget
])
display(engine_widget)

## DC Engine with RC ESC

Most of RC ESC (Electronic Speed Controller) are based in PWM similar used with servos.
Exists alternative RC ESC protocolos (PPM, ...) but Tamiya TBLE 04s uses tipical PWM like Servos.

- **Base Freq**: 50hz
- **Full Reverse**: Pulses of 10ms
- **Neutral**: Pulses of 15ms
- **Full Throthle**: Pulses of 20ms

This is de ESC input protocol. ESC_Output/DC_input is a PWM too but de Duty Cycle is proportional to Speed. Reverse inverts polarity and Neutral is a coninuous 0V signal.

In [6]:
import time
from adafruit_servokit import ServoKit

# Set channels to the number of servo channels on your kit.
# 8 for FeatherWing, 16 for Shield/HAT/Bonnet.
kit = ServoKit(channels=16)

In [4]:
kit.continuous_servo[8].throttle = -1
time.sleep(1)
kit.continuous_servo[8].throttle = 0
time.sleep(1)
kit.continuous_servo[8].throttle = 1

<p float="left">
  <img src="images/pwm_throttle_-1.jpg" alt="angle 0" style="width:30%">
  <img src="images/pwm_throttle_0.jpg" alt="angle 0" style="width:30%">
  <img src="images/pwm_throttle_1.jpg" alt="angle 0" style="width:30%">
</p>

## Test DC Brushed Engine

### DC Brushed Engine: **Mabuchi RS540 SH - 7525**
### ESC: **Tamiya TBLE 04S**

ESC operates at **7V** with a Battery *Cut-Off*. This test is powered with one Standard PC Power Supply. Taking energy from 5V and 12V (stepped down to 7.1V using **AZDelivery XL4016E1** Converter)

Current can be limited too on XL4016E1. Not limiting (or too high limit) causes the PU switch off when the engine demmands full energy. ESC limit is higher than PU capacity. Setting a limit too low at XL4016E1, causes the engine stops safety (dead zones appears in the scroll bar). (Measuring with polymeter at XL4016E1 output, we can see voltaje decreases while current increase)

#### **Calibration**
It's necesary calibrate the ESC to adapt the PWM generated by the controller (or RC receptor)

1. Start the system with the ESC Switch OFF. ESC beeps and green blink until a PWM is received.
2. Init the test. It generates a PWM. ESC stops beeps.
3. With ESC button pressed, switch ON. Led starts blinkin and repeat (RED-GREEN-ORANGE). When led are RED release the button. Now led blinks RED.
4. Generate PWM for full throttle and press-release the button.
5. Generate PWM for full reverse and press-release the button.
6. Done. ESC has been calibrated.

**Reverse** can be enabled/disabled with the steps 1-3.

### **WARNING**
Measuring Voltage on 3-wire PWM output of ESC is 5.9V. This propagates to **PCA9685** and to **RPI/Jetson**
It can be use a Voltage limiter or simpli don't connect red wire. With only GND+Data the system works.

In [1]:
from adafruit_servokit import ServoKit
import time
import ipywidgets as widgets
from traitlets import CInt, link, CFloat

SERVO_CHANNEL = 9

class Counter(widgets.DOMWidget):
    value = CFloat(0)
    
# Value Widget to Sync Sliders and buttons
counter = Counter()    

# Servo Kit
kit = ServoKit(channels=16)

# Set Neutral
kit.continuous_servo[SERVO_CHANNEL].throttle = 0

# GUI
style = {'description_width': 'initial'}
dc_speed_widget = widgets.FloatSlider(description='DC Speed', style=style, min=-1, max=1, step=0.01, value=0, layout=widgets.Layout(width='50%', height='30px'))
dc_reset_widget = widgets.Button(description='DC Reset')
output = widgets.Output()

def on_dc_speed_value_change(change):
    #with output:
        speed = change['new']
        kit.continuous_servo[SERVO_CHANNEL].throttle = speed
        counter.value = speed
        #print('Set Speed : '+str(speed))
        
dc_speed_widget.observe(on_dc_speed_value_change, names='value')

def on_dc_reset_clicked(b):
    #with output:
        counter.value = 0
        print("Speed Reset.")
        
dc_reset_widget.on_click(on_dc_reset_clicked) 

# Link Slider and counter
link((dc_speed_widget, 'value'), (counter, 'value')) # Link counter al slider

engine_widget = widgets.VBox([
    dc_speed_widget,
    dc_reset_widget
])
display(engine_widget)

## **BANRC01** - Test Bank

<div style="text-align:center;"><img src="images/banrc01_testbank.jpg"  width="800"/></div>

#### DC Engine PWM Input
Voltage Signal readed from the DC Engine input at diferent throthle levels (not the PWM sended to ESC).

<table>
    <tr>
        <th>
            <span>throttle: 0.0</span>
            <img src="images/DC-00.jpg" alt="angle 0" style="width:30%">
            <span>throttle: 0.02</span>
            <img src="images/DC-02.jpg" alt="angle 0" style="width:30%">
        </th>
    </tr>
    <tr>
        <th>
            <span>throttle: 0.04</span>
            <img src="images/DC-04.jpg" alt="angle 0" style="width:30%">
            <span>throttle: 0.06</span>
            <img src="images/DC-06.jpg" alt="angle 0" style="width:30%">
        </th>
    </tr>
    <tr>
        <th>
            <span>throttle: 0.08</span>
            <img src="images/DC-08.jpg" alt="angle 0" style="width:30%">
            <span>throttle: 0.10</span>
            <img src="images/DC-10.jpg" alt="angle 0" style="width:30%">
        </th>
    </tr>
    <tr>
        <th>
            <span>throttle: 0.12</span>
            <img src="images/DC-12.jpg" alt="angle 0" style="width:30%">
            <span>throttle: 0.14</span>
            <img src="images/DC-14.jpg" alt="angle 0" style="width:30%">
        </th>
    </tr>
    <tr>
        <th>
            <span>throttle: 0.16</span>
            <img src="images/DC-16.jpg" alt="angle 0" style="width:30%">
        </th>
    </tr>
</table>

### Funny simulation of F1 start

In [1]:
from adafruit_servokit import ServoKit
import time

SERVO_CHANNEL = 9

# Servo Kit
kit = ServoKit(channels=16)

# Set Neutral
kit.continuous_servo[SERVO_CHANNEL].throttle = 0

throttle = 0
step = 0.01
time_step_acc = 0.01
time_step_gain_gear = 0.3
time_step_dec = 0.005
time_step_pit = 0.04

max_throttle = 0.7
min_throttle = 0.08
start_throttle = 0.3
gears = 10

def setThrotlle(t):
    global throttle
    throttle = t
    kit.continuous_servo[SERVO_CHANNEL].throttle = throttle
    #print('Throttle : {:.5f}'.format(throttle))

try:
    # Start Engine
    setThrotlle(min_throttle)
    time.sleep(3)
    setThrotlle(start_throttle)
    time.sleep(2)
    setThrotlle(min_throttle)
    time.sleep(1)
    
    # Gears up
    min_throttle_thres = min_throttle
    for g in range(0, gears):
        print('Gear: {}'.format(g))
        while(throttle <= max_throttle):
            setThrotlle(throttle + step)
            time.sleep(time_step_acc)
            
        #print('time_step_acc : {:.5f}'.format(time_step_acc))
        time_step_acc = time_step_acc + (time_step_acc * time_step_gain_gear)
        if(g < gears):
            setThrotlle(min_throttle_thres)
            min_throttle_thres = min(min_throttle_thres * 1.4, max_throttle * 0.5)

    # Gears down
    for g in range(0, gears):
        print('Gear: {}'.format(g))
        while(throttle >= min_throttle):
            setThrotlle(throttle - step)
            time.sleep(time_step_dec)
        setThrotlle(max_throttle)
      
    # Pit Enter
    setThrotlle(min_throttle)
    time.sleep(5)
    while(throttle >= 0):
        setThrotlle(throttle - step)
        time.sleep(time_step_pit)
        
    setThrotlle(0)
except KeyboardInterrupt:
    setThrotlle(0)

Gear: 0
Gear: 1
Gear: 2
Gear: 3
Gear: 4
Gear: 5
Gear: 6
Gear: 7
Gear: 8
Gear: 9
Gear: 0
Gear: 1
Gear: 2
Gear: 3
Gear: 4
Gear: 5
Gear: 6
Gear: 7
Gear: 8
Gear: 9


In [14]:
setThrotlle(0)

Throttle : 0.00000


## DC Engine Current Consumption

In [35]:
from adafruit_servokit import ServoKit
import time

SERVO_CHANNEL = 9

# Servo Kit
kit = ServoKit(channels=16)

# Set Neutral
kit.continuous_servo[SERVO_CHANNEL].throttle = 0

start=0.3
fromT=0
toT=0.8

try:
    kit.continuous_servo[SERVO_CHANNEL].throttle = 0.3
    time.sleep(0.5)
    #kit.continuous_servo[SERVO_CHANNEL].throttle = fromT
    #time.sleep(5)
    kit.continuous_servo[SERVO_CHANNEL].throttle = toT
    time.sleep(5)
    kit.continuous_servo[SERVO_CHANNEL].throttle = 0
except KeyboardInterrupt:
    kit.continuous_servo[SERVO_CHANNEL].throttle = 0

#### Current with Empty Load

|start-end % | Max Peak      | Stationary |
|------      |---------------|------------|
| 0-0.1      | 0.781 A       | 0.578 A    |
| 0-0.2      |  1.259 A      | 0.889 A    |
| 0-0.3      |  2.324 A      | 0.97  A    |
| 0-0.4      |  2.327 A      | 1.094 A    |
| 0-0.5      |  2.836 A      | 1.199 A    |
| 0-0.6      |   -           | 1.734 A    |
| 0-0.7      |   -           | 1.923 A    |
| 0-0.8      |   -           | 2.036 A    |

From 0-0.6 the DC-DC (i assumme) can not provide enought current peak (AZDelivery Documentation specifies **4A** Max). In practice shuldn't exist this intervals, approaching to the final value progressive. 
If this situation ocurred, we can delegate the power delivery to al algorithm that manages in steps.